In [12]:
import json
import re

def read_and_split_log_file(file_path):
    """
    Read a log file and split it into chunks wherever '}{' is found.
    The '}' goes to the previous chunk and '{' goes to the next chunk.
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        content = content.lstrip('\ufeff')
    
    # Split the content by '}{' pattern
    # This will split at the boundary between JSON objects
    chunks = re.split(r'}{', content)
    
    # Process chunks to properly format them as complete JSON objects
    json_objects = []
    
    for i, chunk in enumerate(chunks):
        chunk = chunk.strip()
        if not chunk:
            continue
            
        # Add opening brace to all chunks except the first
        if i > 0:
            chunk = '{' + chunk
            
        # Add closing brace to all chunks except the last
        if i < len(chunks) - 1:
            chunk = chunk + '}'
        
        # Try to parse as JSON
        try:
            json_obj = json.loads(chunk)
            json_objects.append(json_obj)
        except json.JSONDecodeError as e:
            print(f"Failed to parse chunk {i}: {e}")
            print(f"Chunk content: {chunk[:100]}...")  # Show first 100 chars
        

    return json_objects




In [ ]:
import os

# Define the folder path
folder_path = "/Users/vaibhavmishra/Desktop/btx-game-aicode/clash_squad/683a0a7c45c7d715f24bf50a"

# List to store complete paths of .log files
log_files = []

# Walk through the directory and find all .log files
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith('.log'):
            # Get the complete path of the file
            complete_path = os.path.join(root, file)
            log_files.append(complete_path)

print(f"Found {len(log_files)} .log files:")
# Sort log files by the number in their filename
def extract_number_from_filename(filename):
    """Extract the number from filenames like '500_recorder.log'"""
    import re
    match = re.search(r'(\d+)_recorder\.log', filename)
    return int(match.group(1)) if match else 0

# Sort log_files by the number in their filename
log_files.sort(key=lambda x: extract_number_from_filename(os.path.basename(x)))

print(f"Found {len(log_files)} .log files:")
for log_file in log_files:
    print(f"  {os.path.basename(log_file)}")


print(log_files)
game_json_objects = []
for log_file in log_files:
    try:
        json_objects = read_and_split_log_file(log_file)
        print(f"Successfully parsed {len(json_objects)} JSON objects")
        game_json_objects.extend(json_objects)
        # Display first few objects
        # for i, obj in enumerate(json_objects[:3]):
        #     print(f"\nObject {i+1}:")
        #     print(json.dumps(obj, indent=2))
        
    except Exception as e:
        print(f"Error reading file: {e}")

In [ ]:
print(len(game_json_objects))
# Filter out JSON objects with round_number = 0
game_json_objects = [obj for obj in game_json_objects if obj.get('round_number', 0) != 0]

print(f"After filtering, remaining JSON objects: {len(game_json_objects)}")


In [ ]:
print(game_json_objects[0])

In [ ]:
from collections import defaultdict
import numpy as np


def prepare_dataset(data):
    """
    prepare the dataset from data which is a per game data, 
    get the features as a dictionary with (agent, [ agent_data ] ) where agent is not a bot 

    """
    # Group samples by game_id
    
    game_agents = defaultdict(list)

    for sample in data:
        game_id = sample["game_id"]
        for agent in sample["agents_data"]:
            if agent["isBot"] == False:
                key = f"{game_id}_{agent['agent_id']}"
                game_agents[key].append(agent)
    
    return game_agents

dataset = prepare_dataset(game_json_objects)

for key in dataset.keys():
    print(f"Key: {key}")
    print(f"Value: {dataset[key]}")
    break

In [ ]:
import json

# Convert numpy types to native Python types for JSON serialization
def convert_numpy(obj):
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    return obj

# Save the dataset to a JSON file
with open("clash_squad_dataset.json", "w") as f:
    json.dump(dataset, f, default=convert_numpy)

print("Dataset saved to clash_squad_dataset.json")



In [ ]:
import json

def convert_numpy(obj):
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    return obj

for key in dataset.keys():
    agent_timeline = dataset[key]
    filename = f"{key}.json"
    with open(filename, "w") as f:
        json.dump(agent_timeline, f, default=convert_numpy)
    print(f"Saved {filename}")



PUTTING EVERYTHING TOGETHER

In [3]:
import json
import re
from collections import defaultdict
import numpy as np
import os

def read_and_split_log_file(file_path):
    """
    Read a log file and split it into chunks wherever '}{' is found.
    The '}' goes to the previous chunk and '{' goes to the next chunk.
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        content = content.lstrip('\ufeff')
    
    # Split the content by '}{' pattern
    # This will split at the boundary between JSON objects
    chunks = re.split(r'}{', content)
    
    # Process chunks to properly format them as complete JSON objects
    json_objects = []
    
    for i, chunk in enumerate(chunks):
        chunk = chunk.strip()
        if not chunk:
            continue
            
        # Add opening brace to all chunks except the first
        if i > 0:
            chunk = '{' + chunk
            
        # Add closing brace to all chunks except the last
        if i < len(chunks) - 1:
            chunk = chunk + '}'
        
        # Try to parse as JSON
        try:
            json_obj = json.loads(chunk)
            json_objects.append(json_obj)
        except json.JSONDecodeError as e:
            print(f"Failed to parse chunk {i}: {e}")
            print(f"Chunk content: {chunk[:100]}...")  # Show first 100 chars
        

    return json_objects

def extract_number_from_filename(filename):
    """Extract the number from filenames like '500_recorder.log'"""
    import re
    match = re.search(r'(\d+)_recorder\.log', filename)
    return int(match.group(1)) if match else 0


def prepare_dataset(data):
    """
    prepare the dataset from data which is a per game data, 
    get the features as a dictionary with (agent, [ agent_data ] ) where agent is not a bot 

    """
    # Group samples by game_id
    
    game_agents = defaultdict(list)

    for sample in data:
        game_id = sample["game_id"]
        for agent in sample["agents_data"]:
            if agent["isBot"] == False:
                key = f"{game_id}_{agent['agent_id']}"
                game_agents[key].append(agent)
    
    return game_agents


def convert_numpy(obj):
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    return obj


In [ ]:


# Define the folder path
import os

folder_path = "/Users/vaibhavmishra/Desktop/btx-game-aicode/clash_squad/"
subfolders = [f.path for f in os.scandir(folder_path) if f.is_dir()]
print("Subfolders:", subfolders)

# List to store complete paths of .log files

for game_folder in subfolders:
    log_files = []
    # Walk through the directory and find all .log files
    # Combine folder_path and game_folder to get the full path to the game folder
    full_game_folder_path = os.path.join(folder_path, os.path.basename(game_folder))
    for root, dirs, files in os.walk(full_game_folder_path):
        for file in files:
            if file.endswith('.log'):
                # Get the complete path of the file
                complete_path = os.path.join(root, file)
                log_files.append(complete_path)

    print(f"Found {len(log_files)} .log files:")
    # Sort log files by the number in their filename
    
    # Sort log_files by the number in their filename
    log_files.sort(key=lambda x: extract_number_from_filename(os.path.basename(x)))

    print(log_files)
    game_json_objects = []
    for log_file in log_files:
        try:
            json_objects = read_and_split_log_file(log_file)
            print(f"Successfully parsed {len(json_objects)} JSON objects")
            game_json_objects.extend(json_objects)
            
        except Exception as e:
            print(f"Error reading file: {e}")


    print(len(game_json_objects))
    # Filter out JSON objects with round_number = 0
    game_json_objects = [obj for obj in game_json_objects if obj.get('round_number', 0) != 0]

    print(f"After filtering, remaining JSON objects: {len(game_json_objects)}")






    dataset = prepare_dataset(game_json_objects)

    for key in dataset.keys():
        print(f"Key: {key}")
        print(f"Value: {dataset[key]}")
        break



    for key in dataset.keys():
        agent_timeline = dataset[key]
        filename = f"/Users/vaibhavmishra/Desktop/btx-game-aicode/clash_squad_agent_partition/{key}.json"
        with open(filename, "w") as f:
            json.dump(agent_timeline, f, default=convert_numpy)
        print(f"Saved {filename}")
     
    


